<a href="https://colab.research.google.com/github/anespart1/educationuse/blob/main/%ED%8B%B0%EC%B2%98%EB%B8%94%EB%A8%B8%EC%8B%A0_%ED%99%9C%EC%9A%A9%EC%9D%84_%EC%9C%84%ED%95%9C_%EC%9D%B4%EB%AF%B8%EC%A7%80%ED%81%AC%EB%A1%A4%EB%9F%AC_%EC%A1%B0%EA%B1%B4%EC%84%A0%ED%83%9D%ED%98%95_20220830.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#셀레니움 설치 / 가져오기 / 크롬드라이버 준비(숨김)

In [1]:
#처음 실행 때 설치를 위해 필요함
!pip install selenium==4.2.0 #셀레니움이 업데이트 되면서 사용방법이 달라짐. 최대한 과거와 비슷하게 맞추었지만 일부 메소드가 수정됨
!apt-get update
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 983 kB 9.3 MB/s 
     |████████████████████████████████| 358 kB 46.4 MB/s 
     |████████████████████████████████| 140 kB 45.0 MB/s 
     |████████████████████████████████| 4.1 MB 47.9 MB/s 
     |████████████████████████████████| 55 kB 3.8 MB/s 
     |████████████████████████████████| 58 kB 4.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://develo

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By # 이거 없으면 By.~ 부분 에러남
from numpy.random.mtrand import noncentral_chisquare
import time, random, os, shutil
import urllib.request
import numpy as np

In [3]:
def web_image_selector(keyword=None, url = "https://www.google.co.kr/imghp?hl=ko&tab=ri&authuser=0&ogbl"):
  #옵션 설정 및 크롬 드라이버 준비
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')

  #디렉토리 경로 초기화
  path = '/content'
  try:
    os.mkdir('{}/{}'.format(path, 'download'))
  except FileExistsError:
    pass
  os.chdir('{}/{}'.format(path, 'download'))
  #드라이버 시동, URL 열기
  driver = webdriver.Chrome('chromedriver', options=chrome_options)
  driver.get(url)
  #구글 웹 엘리먼트 클래스값(q)을 활용, 검색어 입력
  search = driver.find_element(by=By.NAME, value='q')
  search.send_keys(keyword)
  search.send_keys(Keys.RETURN)
  
  #스크롤 내리기(최대)
  last_height = driver.execute_script("return document.body.scrollHeight")

  while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
      break
    last_height = new_height

  #CSS 태그로 선택해서 리스트 작성
  images = driver.find_elements(by=By.CSS_SELECTOR, value=".rg_i.Q4LuWd")
  return driver, images, keyword, path

In [4]:
def image_downloader(img_check=None, driver=None, img=None, i=None, keyword=None, count=None):
  driver.execute_script("arguments[0].click();",img)
  time.sleep(2)
  
  #큰이미지 주소 호출
  if img_check == img:
    target = driver.find_elements(By.CSS_SELECTOR,"img.n3VNCb")[0]
  else:
    target = driver.find_elements(By.CSS_SELECTOR,"img.n3VNCb")[1]

  img_src = target.get_attribute('src')
  
  #이미지 다운로드
  urllib.request.urlretrieve(img_src, keyword + '_' + str(i+1) + '_' + str(count) + ".jpg")

In [5]:
def web_image_crawler(keyword=None, select_range=None, start_range=None, end_range=None,  select_method=None, images=None, stop=None):
  #1. 저장경로 생성
  try:
    os.mkdir('{}/{}/{}'.format(path, 'download', keyword))
  except FileExistsError:
    pass
  os.chdir('{}/{}/{}'.format(path, 'download', keyword))

  #https://www.inflearn.com/chats/530544 참고 작성
  #2. 조건에 따른 작업 분기점 생성
  tot_nums = list(range(len(images))) #전체 이미지링크수 리스트
  count = 0 #기본적인 카운터
  img_check = images[0]

  if select_method == "뒤에서부터":
    if select_range =="네":
      for n, img in zip(sorted(tot_nums[start_range:end_range], reverse=True), reversed(images[start_range:end_range])):
        if count < stop:
          try:
            image_downloader(driver=driver, img=img, keyword=keyword, i=n, count=count)
            count = count + 1
            print(str(keyword) + " 이미지를 가져왔습니다")
          except:
            pass
        else:
          break

    else: #select_range == 아니오 의 경우
      for n, img in zip(sorted(tot_nums, reverse=True), reversed(images)):
        if count < stop:
          try:
            image_downloader(driver=driver, img=img, keyword=keyword, i=n, count=count)
            count = count + 1
            print(str(keyword) + " 이미지를 가져왔습니다")
          except:
            pass
        else:
          break
          

  #홀수번째만의 경우(0부터 1번째이므로, 0이 남는 게 홀수임)
  elif select_method == "홀수번째만":
    if select_range == "네":
      for n, img in zip(tot_nums[start_range:end_range], images[start_range:end_range]):
        if count < stop:
          if n%2 == 0:
            try:
              image_downloader(driver=driver, img=img, keyword=keyword, i=n, count=count)
              count = count + 1
              print(str(keyword) + " 이미지를 가져왔습니다")
            except:
              pass
        else:
          break

    else: #select_range == 아니오 의 경우
      for n, img in zip(tot_nums, images):
        if count < stop:
          if n%2 == 0:
            try:
              image_downloader(driver=driver, img=img, keyword=keyword, i=n, count=count)
              count = count + 1
              print(str(keyword) + " 이미지를 가져왔습니다")
            except:
              pass
        else:
          break


  #짝수번째만의 경우(0부터 1번째이므로, 1이 남는 게 짝수임)
  elif select_method == "짝수번째만":
    if select_range == "네":
      for n, img in zip(tot_nums[start_range:end_range], images[start_range:end_range]):
        if count < stop:
          if n%2 == 1:
            try:
              image_downloader(driver=driver, img=img, keyword=keyword, i=n, count=count)
              count = count + 1
              print(str(keyword) + " 이미지를 가져왔습니다")
            except:
              pass
        else:
          break

    else: #select_range == 아니오 의 경우
      for n, img in zip(tot_nums, images):
        if count < stop:
          if n%2 == 1:
            try:
              image_downloader(driver=driver, img=img, keyword=keyword, i=n, count=count)
              count = count + 1
              print(str(keyword) + " 이미지를 가져왔습니다")
            except:
              pass
        else:
          break


  #무작위선택의 경우
  elif select_method == "무작위선택":
    if select_range == "네":
      num_box = list(range(start_range, end_range))
      check_box = random.sample(num_box, len(num_box))
      for n in check_box: #안되는 링크로 인한 손실을 피하기 위해, 랜덤 셔플 후 순차적으로 선별하는 방식으로 함
        img = images[n]
        if count < stop:
          try:
            image_downloader(driver=driver, img=img, keyword=keyword, i=n, count=count)
            count = count + 1
            print(str(keyword) + " 이미지를 가져왔습니다")
          except:
            pass
        else:
          break

    else: #select_range == 아니오 의 경우
      num_box = list(range(images))
      check_box = random.sample(num_box, len(num_box))
      for n in check_box: #안되는 링크로 인한 손실을 피하기 위해, 랜덤 셔플 후 순차적으로 선별하는 방식으로 함
        img = images[n]
        if count < stop:
          try:
            image_downloader(driver=driver, img=img, keyword=keyword, i=n, count=count)
            count = count + 1
            print(str(keyword) + " 이미지를 가져왔습니다")
          except:
            pass
        else:
          break


  else: #select_method == "앞에서부터"의 경우
    if select_range =="네":
      for n, img in zip(tot_nums[start_range:end_range], images[start_range:end_range]):
        if count < stop:
          try:
            image_downloader(driver=driver, img=img, keyword=keyword, i=n, count=count)
            count = count + 1
            print(str(keyword) + " 이미지를 가져왔습니다")
          except:
            pass
        else:
          break

    elif select_range == "아니오":
      for n, img in zip(tot_nums, images):
        if count < stop:
          try:
            image_downloader(driver=driver, img=img, keyword=keyword, i=n, count=count)
            count = count + 1
            print(str(keyword) + " 이미지를 가져왔습니다")
          except:
            pass
        else:
          break


  #결과 보고
  print(str(keyword) + " 이미지(총 {}개를) 가져왔습니다. finished.".format(count))
  driver.close()

In [6]:
a = [1,2,3,4,5]
a.pop(1)
print(a)

[1, 3, 4, 5]


#구글 이미지크롤링 및 저장(입력후 Ctrl+F10)

In [7]:
#@markdown ----
#@markdown ### 카페인이 함유된 식품을 입력해주세요:
#@markdown (cf.초콜릿 아이스크림 콜라) 구분은 띄어쓰기로만 부탁드립니다.
keywords = "\uC544\uC774\uB9AC\uC2A4 \uBAA8\uB798\uC2DC\uACC4" #@param {type:"string"}
#@markdown ----
#@markdown ### 이미지크롤링 구간을 선택합니까?:
select_range = "\uB124" #@param ["네", "아니오"] {type:"string"}
#@markdown 아래의 슬라이드를 움직여 구간을 정해주세요
start_range = 22 #@param {type:"slider", min:0, max:400, step:1}
end_range = 58 #@param {type:"slider", min:0, max:400, step:1}
#@markdown ----
#@markdown ### 크롤링 조건을 설정해주세요:
#@markdown (앞에서부터, 뒤에서부터, 홀수번째만, 짝수번째만, 무작위선택)
select_method = "\uBB34\uC791\uC704\uC120\uD0DD" #@param ["앞에서부터", "뒤에서부터", "홀수번째만", "짝수번째만", "무작위선택"] {type:"string"}
#@markdown ----
#@markdown ### 이미지크롤링 개수를 정해주세요:
#@markdown (cf. 30 = 초콜릿 30개, 아이스크림 30개, 콜라 30개)
how_many = 10 #@param {type:"slider", min:10, max:200, step:10}
#@markdown ----
for kword in keywords.split():
  driver, images, keyword, path = web_image_selector(keyword=kword)
  web_image_crawler(keyword=keyword, select_range=select_range,
                    start_range=start_range, end_range=end_range,
                    select_method=select_method, images=images, stop=how_many)

#파일 다운로드
!zip -r /content/download.zip /content/download/

from google.colab import files
files.download('/content/download.zip')

아이리스 이미지를 가져왔습니다
아이리스 이미지를 가져왔습니다
아이리스 이미지를 가져왔습니다
아이리스 이미지를 가져왔습니다
아이리스 이미지를 가져왔습니다
아이리스 이미지를 가져왔습니다
아이리스 이미지를 가져왔습니다
아이리스 이미지를 가져왔습니다
아이리스 이미지를 가져왔습니다
아이리스 이미지를 가져왔습니다
아이리스 이미지(총 10개를) 가져왔습니다. finished.
모래시계 이미지를 가져왔습니다
모래시계 이미지를 가져왔습니다
모래시계 이미지를 가져왔습니다
모래시계 이미지를 가져왔습니다
모래시계 이미지를 가져왔습니다
모래시계 이미지를 가져왔습니다
모래시계 이미지를 가져왔습니다
모래시계 이미지를 가져왔습니다
모래시계 이미지를 가져왔습니다
모래시계 이미지를 가져왔습니다
모래시계 이미지(총 10개를) 가져왔습니다. finished.
  adding: content/download/ (stored 0%)
  adding: content/download/모래시계/ (stored 0%)
  adding: content/download/모래시계/모래시계_52_0.jpg (deflated 0%)
  adding: content/download/모래시계/모래시계_50_5.jpg (deflated 2%)
  adding: content/download/모래시계/모래시계_53_9.jpg (deflated 0%)
  adding: content/download/모래시계/모래시계_28_1.jpg (deflated 0%)
  adding: content/download/모래시계/모래시계_43_7.jpg (deflated 0%)
  adding: content/download/모래시계/모래시계_26_6.jpg (deflated 16%)
  adding: content/download/모래시계/모래시계_38_8.jpg (deflated 0%)
  adding: content/download/모래시계/모래시계_41_3.jpg (deflated 2%)
  adding: content/downlo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#다운로드 폴더, 파일, 모든 변수 삭제(바로 재사용 가능하게 해줌)
shutil.rmtree('/content/download/')
os.remove('/content/download.zip')
%reset -f